Item 45 Consider @property Instead of Refactoring Attributes 

Things to Remember
- Use @property to give existing instance attributes new functionality
- Make incremental progress toward better data models by using @property
- Consider refactoring a class and all call sites when you find yourself using @property too heavily   

One advanced but common use of @property
- transits what was a simple numerical attribute into an on-the-fly calculation
    - allows you to migrate all existing usage of a class to have new behaviors without requiring any of the call sites to be rewritten 

In [ ]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f'Bucket(quota={self.quota})'

In [ ]:
# - the leaky bucket algorithm works by ensuring that,
#   whenever the bucket is filled, the amount of quota
#   does not carry over from one period into the next
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0 # does not carry over the amount
        bucket.reset_time = now
    bucket.quota += amount


In [ ]:
# quota consumer
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False # bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False # bucket was filled, but not enough
    bucket.quota -= amount
    return True     

In [ ]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Problems with the above implementation
- You don't know what quota level the bucket started with
- the caller of deduct doesn't know it's being blocked (not enough quota) because the bucket ran out of quota or the bucket never had quota during this period 

Solution
- introduce two new attributes max_quota and quota_consumed
- use @property to make sure the quota attribute is compatible with the existing code

In [ ]:
from datetime import datetime, timedelta

# - this implementaion assumes you fill up the 
#   bucket only once at most in any given period 
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0 # new attribute
        self.quota_consumed = 0 # new attribute

    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_consumed={self.quota_consumed})')
    @property
    def quota(self): # compute the quota on-the-fly
        return self.max_quota - self.quota_consumed
    
    # - review how the fill method 
    #   is implemented to understand 
    #   why the logic here is 
    #   implemented this way  
    @quota.setter
    def quota(self, amount): # amount is the new amount for quota
        delta = self.max_quota - amount
        # print(f'delta: {delta} amount:{amount}')
        if amount == 0: # - fill method will set 
                        #   the quota to zero when
                        #   the period expires   
            # simulating resetting quota to 0
            self.quota_consumed = 0
            self.max_quota = 0
        # - you are filling the bucket the first
        #   time in a given period
        elif delta < 0:
            # - make sure it's a new period
            #   before setting up the max_quota 
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed = delta 

In [ ]:
# - notice that we don't need to modify the fill
#   or the deduct methods even though the quota
#   attribute has been completely reimplemented
bucket = NewBucket(60)
fill(bucket, 100)
print(bucket)
deduct(bucket, 50)
print(bucket)
deduct(bucket, 40)
print(bucket)
assert bucket.quota == 10 
assert deduct(bucket, 20) == False # can't consume another 20

Benefits of using @property
- the code using Bucket.quota doesn't have to change or know that the class has changed.
- allows you to make incremental progress toward a better data model over time.  

About the Bucket implementation
- looks like fill and deduct should have been implemented as instance methods of the class
- this is to showcase that in practice there are many situations in which objects start with poorly defined interfaces or ac as dumb data containers
- @property is a tool to help you address problems in real-world code. Don't overuse it.
- When you find yourself repeatedly extending @property methods, it's probably time to refactor your class instead of further paving over your code's poor design  